# WESAD Validation Notebook for FLIRT


In [1]:
# Import Packages
import pandas as pd
import numpy as np

import matplotlib; matplotlib.use('agg')
import matplotlib.pyplot as plt

import multiprocessing
from joblib import Parallel, delayed
from tqdm.autonotebook import trange

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import utils, model_selection, metrics

from datetime import datetime, timedelta, timezone
from typing import List
import lightgbm as lgb
import glob2
import os 

import sys
sys.path.insert(1, '/home/fefespinola/ETHZ_Fall_2020/flirt-1')
import flirt.simple

In [2]:
The following function retrieves all HRV, EDA and ACC features per subject using the FLIRT pipeline


SyntaxError: invalid syntax (<ipython-input-2-c9c2e5945b79>, line 1)

In [2]:
def get_features_per_subject(path, window_length):
    features = flirt.simple.get_features_for_empatica_archive(zip_file_path = path,
                                      window_length = window_length,
                                      window_step_size = 0.25,
                                      hrv_features = True,
                                      eda_features = True,
                                      acc_features = True,
                                      bvp_features = True,
                                      temp_features = True,
                                      debug = True)
    return features

The following function determines the time offsets of the start and end of each relevant analysis period (baseline, stress, amusement). These offsets are combined with the timestamp stating the start of recording, to determine the absolute timestamps of the sections of interest for each subject. 

In [3]:
def find_label_timestamps(csv_path, StartingTime):

    ID = csv_path.split('/', 3)[2]
    df_timestamp = pd.read_csv(glob2.glob('project_data/WESAD/' + ID + '/*quest.csv')[0], delimiter = ';', header = 1).iloc[:2, :].dropna(axis = 1)
    print('===================================')
    print('Printing the timestamp for {0}'.format(ID))
    print('===================================')
    print(df_timestamp.head())
    
    # Start/End of experiment periods
    print('\nStart of the baseline: ' + str(df_timestamp['Base'][0]))
    print('End of the baseline: ' + str(df_timestamp['Base'][1]))
    print('Start of the fun: ' + str(df_timestamp['Fun'][0]))
    print('End of the fun: ' + str(df_timestamp['Fun'][1]))
    print('Start of the stress: ' + str(df_timestamp['TSST'][0]))
    print('End of the stress: ' + str(df_timestamp['TSST'][1]))
    
    # Get start and end time and assign label into a dict
    lab_dict = {'Base':0, 'TSST':1, 'Fun':2}
    labels_times_dict = {}
    for mode in df_timestamp.columns.tolist():
        print('mode', mode)
        if mode=='Base' or mode=='Fun' or mode=='TSST':
            labels_times_dict[mode] = [StartingTime + timedelta(minutes = int(str(df_timestamp[mode][0]).split(".")[0]))+ timedelta                                         (seconds = int(str(df_timestamp[mode][0]).split(".")[1])), 
                                    StartingTime + timedelta(minutes = int(str(df_timestamp[mode][1]).split(".")[0])) + timedelta                                           (seconds = int(str(df_timestamp[mode][1]).split(".")[1])), lab_dict[mode]]
            
            #labels_times_dict[mode] = [StartingTime + timedelta(minutes = float(df_timestamp[mode][0])), 
                                  #StartingTime + timedelta(minutes = float(df_timestamp[mode][1])), lab_dict[mode]]
        
    return labels_times_dict

In [4]:
import ast
from dateutil.parser import parse

def find_label_start_time(csv_path):
    ID = csv_path.split('/', 3)[2]
    timestamp = open(glob2.glob('project_data/WESAD/' + ID + '/*respiban.txt')[0], "r")
    for i in range(2):
        line = (timestamp.readline())
        line = line.strip()[2:]
        if i==1:
            dict = ast.literal_eval(line)
            start_time_str = dict['00:07:80:D8:AB:58']['time']
            date_str = dict['00:07:80:D8:AB:58']['date']
            datetime_str = date_str + " " + start_time_str
            #print(datetime_str)
            date_time_obj = parse(datetime_str)
            #print(date_time_obj)
            start_time = date_time_obj
            utc_time = start_time - timedelta(hours=2)

    timestamp.close()

    #df_timestamp = pd.read_table(glob2.glob('project_data/WESAD/' + ID + '/*respiban.txt')[0], delim_whitespace=True)#.iloc[:2, :].dropna(axis = 1)
    print('===================================')
    print('Printing the timestamp for {0}'.format(ID))
    print('===================================')
    #print(df_timestamp.head())
    return utc_time

Plots the training and validation classification metric evolution with the number of iterations. 

In [5]:
os.chdir('/home/fefespinola/ETHZ_Fall_2020/') #local directory where the script is
File_Path = glob2.glob('project_data/WESAD/**/*_readme.txt', recursive=True)
for subject_path in File_Path:
    start_time = find_label_start_time(subject_path)
    print(start_time)

Printing the timestamp for S10
2017-07-25 07:25:01
Printing the timestamp for S11
2017-07-25 11:33:01
Printing the timestamp for S13
2017-08-08 11:33:01
Printing the timestamp for S14
2017-08-09 07:31:01
Printing the timestamp for S15
2017-08-10 07:30:01
Printing the timestamp for S16
2017-08-10 12:21:01
Printing the timestamp for S17
2017-08-11 07:39:01
Printing the timestamp for S2
2017-05-22 07:39:01
Printing the timestamp for S3
2017-05-24 11:26:01
Printing the timestamp for S4
2017-06-13 08:57:01
Printing the timestamp for S5
2017-06-13 12:42:01
Printing the timestamp for S6
2017-06-14 11:40:01
Printing the timestamp for S7
2017-07-06 11:30:01
Printing the timestamp for S8
2017-07-10 11:28:01
Printing the timestamp for S9
2017-07-11 11:26:01


In [6]:
def render_metric(eval_results, metric_name):
    ax = lgb.plot_metric(evals_result, metric=metric_name, figsize=(10, 5))
    #plt.show()
    plt.savefig('/home/fefespinola/ETHZ_Fall_2020/plots/render_metric_all_ekf_feat.png')

Plots the 10 top important classification features, i.e. the ones that influence the output the most.

In [7]:
def render_plot_importance(gbm, importance_type, max_features=10, ignore_zero=True, precision=3):
    ax = lgb.plot_importance(gbm, importance_type=importance_type,
                             max_num_features=max_features,
                             ignore_zero=ignore_zero, figsize=(12, 8),
                             precision=precision)
    #plt.show()
    plt.savefig('/home/fefespinola/ETHZ_Fall_2020/plots/feature_importance_all_ekf_feat.png')


Main function that calls the above functions, determines the relevant data to use (i.e. that within the useful recording periods of baseline, stress and amusement) using the timestampp offsets, assignes the appropriate label to each sample and returns the full data with training samples and the corresponding labels.

In [8]:
def main():
    os.chdir('/home/fefespinola/ETHZ_Fall_2020/') #local directory where the script is
    df_all = pd.DataFrame(None)
    #relevant_features = pd.DataFrame(None)
    File_Path = glob2.glob('project_data/WESAD/**/*_readme.txt', recursive=True)
    window_length = 60 # in seconds
    window_shift = 0.25 # in seconds
    for subject_path in File_Path:
        print(subject_path)
        print(subject_path.split('/', 3)[2])
        ID = subject_path.split('/', 3)[2]
        zip_path = glob2.glob('project_data/WESAD/' + ID + '/*_Data.zip')[0]
        print(zip_path)
        features = get_features_per_subject(zip_path, window_length)
        features.index.name = 'timedata'
        E4Time = features.index[0]
        print(E4Time)
        StartingTime = find_label_start_time(subject_path)
        print(StartingTime)
        labels_times = find_label_timestamps(subject_path, StartingTime)
        #features.index.tz_localize(tz='UTC')
        relevant_features = features.loc[
            ((features.index.tz_localize(tz=None) >= labels_times['Base'][0]) & (features.index.tz_localize(tz=None) <= labels_times['Base'][1])) 
            | ((features.index.tz_localize(tz=None) >= labels_times['Fun'][0]) & (features.index.tz_localize(tz=None) <= labels_times['Fun'][1])) 
            | ((features.index.tz_localize(tz=None) >= labels_times['TSST'][0]) & (features.index.tz_localize(tz=None) <= labels_times['TSST'][1]))]

        relevant_features.insert(0, 'ID', ID)
        relevant_features['label'] = np.zeros(len(relevant_features))
        relevant_features.loc[(relevant_features.index.tz_localize(tz=None)>=labels_times['Fun'][0]) &
                                (relevant_features.index.tz_localize(tz=None)<=labels_times['Fun'][1]), 'label'] = labels_times['Fun'][2]
        relevant_features.loc[(relevant_features.index.tz_localize(tz=None)>=labels_times['TSST'][0]) & 
                            (relevant_features.index.tz_localize(tz=None)<=labels_times['TSST'][1]), 'label'] = labels_times['TSST'][2]

        # concatenate all subjects and add IDs
        df_all = pd.concat((df_all, relevant_features))
    
    print(df_all)

    return df_all

This function generates and saves feature matrices for the individual physiological signals 

In [9]:
def __get_subset_features(df_all, feature_name: str, eda_method:str='lpf'):
    
    if feature_name=='physio':
        small_df = df_all.loc[:, df_all.columns.str.startswith('hrv')&df_all.columns.str.startswith   ('eda')&df_all.columns.str.startswith('bvp')&df_all.columns.str.startswith('temp')]
        filename = 'features_all_' + features_name +'_' + eda_method + '_feat.csv'
    else:
        small_df = df_all.loc[:, df_all.columns.str.startswith(feature_name)]
        if feature_name=='eda':
            filename = 'features_all_' + features_name +'_' + eda_method + '_feat.csv'
        else:
            filename = 'features_all_' + features_name + '_feat.csv'
    small_df.to_csv(filename)


The following function retrieves the correct training and testing data for LOSO cross-validation. It also deals with missing data (inf and nan), and scales the features.

In [10]:
def __get_train_valid_data(df_all, cv_subject):
    scaler = StandardScaler()

    #training data
    X_train = df_all.loc[df_all['ID']!=cv_subject]  # 500 entities, each contains 10 features
    X_train = X_train.iloc[:, 1:len(df_all.columns)-1]
    X_train = X_train.replace([np.nan, np.inf, -np.inf], -1000)
    X_train = scaler.fit_transform(X_train)
    y_train = df_all.loc[df_all['ID']!=cv_subject, ['label']]  # binary target
    train_data = lgb.Dataset(X_train, label=y_train)

    #validation data
    X_test = df_all.loc[df_all['ID']==cv_subject]  # 500 entities, each contains 10 features
    X_test = X_test.iloc[:, 1:len(df_all.columns)-1]
    X_test = X_test.replace([np.nan, np.inf, -np.inf], -1000)
    X_test = scaler.transform(X_test)
    y_test = df_all.loc[df_all['ID']==cv_subject, ['label']]   # binary target
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

    return X_train, y_train, train_data, X_test, y_test, test_data
 

Run the evaluation script to retrieve the labeled data and train classifier to output f1-score

In [11]:
if __name__ == '__main__':
    df_all = main()
    df_all.to_csv('features_all_all_eda_ekf_leda_Newlabel.csv')
    df_all = pd.read_csv('/home/fefespinola/ETHZ_Fall_2020/features_all_all_eda_ekf_leda_Newlabel.csv')
    df_all.set_index('timedata', inplace=True)
    df_all = df_all.loc[:, ~df_all.columns.str.startswith('hrv')]
    print(df_all)
    ID=df_all.ID
    print(ID.unique())

    ### for binary classification uncomment line below 
    #df_all['label'].replace(2, 0, inplace=True)
    #print('===== BINARY ====')

    print('---start classification---')

    df = df_all.replace([np.inf, -np.inf], np.nan) # np.inf leads to problems with some techniques

    # Clean columns that contain a lot of nan values 
    print(len(df), len(df.columns))
    df = df.dropna(axis=1, thresh=int(len(df)*0.8))
    print(len(df), len(df.columns))
    print('Columns dropped: ', df_all.drop(df.columns, axis=1).columns.values)

    stats = []

    cv = model_selection.LeaveOneGroupOut()

    X = df.drop(columns=['label', 'ID'])
    y = df['label'].astype('int')
    groups = df['ID']
    print("running %d-fold CV..." % (cv.get_n_splits(X, y, groups)))

    mean_fpr = np.linspace(0, 1, 100)
    fig, ax = plt.subplots(figsize=(15, 15))

    for train_index, test_index in cv.split(X, y, groups):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        #### for binary classification uncomment line below
        #params = {'objective': 'binary', 'is_unbalance': True}
        params = {'objective': 'multiclass', 'is_unbalance': True}
        model = lgb.LGBMClassifier(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        stats.append({
            'f1': metrics.f1_score(y_test, y_pred, average="macro")})
            
        #print(metrics.classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

    stats = pd.DataFrame(stats)
    print(stats.f1.mean())

    '''
    #parameters
    param = {'metric': 'auc_mu', 'learning_rate': 0.01, 'num_leaves': 31, 'is_unbalance':True,
        'verbose': 1, 'objective':'multiclass', 'num_class':3, 'lambda_l1':0, 'force_col_wise':True}
    
    subjects = ['S2', 'S3', 'S4', 'S10', 'S11', 'S13', 'S14', 'S15', 'S16', 'S17']

    ##### Start Classification
    f1_tot = 0
    f1_dict = {}
    #data with validation set
    for subj in subjects:
        # get data
        print('===Training for LOSO ', subj, '===')
        _, _, train_data, X_test, y_test, test_data = __get_train_valid_data(df_all, subj)
    
        evals_result = {}  # to record eval results for plotting
        
        #train normally
        bst_norm = lgb.train(param, train_data, num_boost_round=550, valid_sets=[train_data, test_data],                        evals_result=evals_result, verbose_eval=10)

        best_iteration = np.argmax(evals_result['valid_1']['auc_mu'][2:]) + 1
        print ('best iter', best_iteration)
        y_pred = bst_norm.predict(X_test, num_iteration=best_iteration)
        y_pred = np.argmax(y_pred, axis=1)

        f1_metric = f1_score(y_test, y_pred, average='macro')
        f1_dict[subj] = f1_metric
        print(f1_dict)
        f1_tot = f1_tot + f1_metric
        print(confusion_matrix(y_test, y_pred))


    f1_tot = f1_tot/len(subjects)
    print(f1_tot)
    #render_metric(evals_result, param['metric'])
    #render_plot_importance(bst_norm, importance_type='split')
    '''

project_data/WESAD/S10/S10_readme.txt
S10
project_data/WESAD/S10/S10_E4_Data.zip
Reading files
Calculating HRV features
Cleaning data...
Calculate td features



Calculate fd features



Calculate stat features



Calculating EDA features



Calculating ACC features


...


Calculating EDA features



Calculating ACC features



Calculating BVP features



Calculating temperature features



2017-07-11 11:11:41+00:00
Printing the timestamp for S9
2017-07-11 11:26:01
Printing the timestamp for S9
   # ORDER   Base   TSST  Medi 1    Fun  Medi 2  bRead  sRead  fRead
0  # START   1.48  31.45   59.02  68.11   78.45  22.16  46.49   75.3
1    # END  21.48  42.50   66.00  74.43   85.40  23.36  48.11   76.5

Start of the baseline: 1.48
End of the baseline: 21.48
Start of the fun: 68.11
End of the fun: 74.43
Start of the stress: 31.45
End of the stress: 42.5
mode # ORDER
mode Base
mode TSST
mode Medi 1
mode Fun
mode Medi 2
mode bRead
mode sRead
mode fRead
                                   ID  hrv_hrv_mean_nni  hrv_hrv_median_nni  \
timedata                                                                      
2017-07-25 07:27:06+00:00         S10        638.976476          632.251877   
2017-07-25 07:27:06.250000+00:00  S10        639.185703          632.785956   
2017-07-25 07:27:06.500000+00:00  S10        639.971309          635.787627   
2017-07-25 07:27:06.750000+00:00  S10  

# Get relevant features
